In [1]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)


from functions import Separate_data, clean_table_mp  
from functions import read_and_check_input_values 
from functions import create_matrix_A_and_C,format_constraints_elements
from functions import format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import optimize_with_correction
from functions import  construct_result_dataframe,export_result,save_errors,remove_old_recipes

In [2]:
def create_optimal_recipe(recette,table_mp, mp_constraints, elmt_quality_constraints,dossier_data):
    df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]

    df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)
    # Suppression des matières premières indisponibles dans
    table_mp = clean_table_mp(table_mp, df_mp_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)
    
    # Initialisation des listes pour les contraintes
    constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

    # Mettre  les contraintes concernant les éléments
    constraints = format_constraints_elements(df_contraints_element, A,constraints)

    # Mettre  les contraintes concernant la qualité
    constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

    # Mettre les contraintes concernant les matières premières disponibles
    constraints, bounds = format_constraints_MP(df_mp_dispo, constraints)

    # Résoudre le problème d'optimisation linéaire
    method = 'simplex' #'interior-point' 'simplex'
    coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    # res, erreurs = optimize_with_correction(table_mp, C, constraints, bounds, method, coefficients)
    res, erreurs = solve_linear_program(C, constraints, bounds,method)
    
    # Gestion du recettes
    if not erreurs: 
        # Construire le DataFrame résultats
        df_res,contraints_res = construct_result_dataframe(res, df_mp_dispo, table_mp,constraints)
        # Écrire le DataFrame résultats dans le fichier Excel
        export_result(df_res, dossier_data, new_sheet_name= recette)
        print("Le problème pour la recette" ,recette,"admet une solution.")

    else :
        # Sauvegarder les erreurs dans un fichier texte
        save_errors(erreurs, dossier_data,recette)
        print("Les erreurs du problème " ,recette," ont été enregistrées dans un fichier")
    return 

In [3]:

if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)
    # Suppression du vieux resultats
    remove_old_recipes(dossier_data)
    # Resolutions du nouveau probleme
    Recettes = [col for col in pd.read_excel(chemin_fichier, engine='calamine', sheet_name=2).columns if 'Unnamed' not in col]
    for recette in tqdm(Recettes, desc="Processing recipes", unit="recipe"):
        table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_input_values(chemin_fichier)
        if errors :
            save_errors(errors, dossier_data,recette)
            break;
        create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_data)


Processing recipes:  50%|█████     | 2/4 [00:00<00:00,  6.56recipe/s]

Le problème pour la recette GS 400-15 admet une solution.
Le problème pour la recette GS 450-10 admet une solution.


Processing recipes: 100%|██████████| 4/4 [00:00<00:00,  9.33recipe/s]

Les erreurs du problème  GS 500-7  ont été enregistrées dans un fichier
Les erreurs du problème  GS 600-3  ont été enregistrées dans un fichier
